<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/4.3.%20%E8%87%AA%E5%AE%9A%E4%B9%89%E5%B1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##4.3. 自定义层
&emsp;&emsp;深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然keras提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何使用keras来自定义一个层，从而可以被重复调用。

###4.3.1. 不含模型参数的自定义层
&emsp;&emsp;我们先介绍如何定义一个不含模型参数的自定义层。这个层里不含模型参数。下面的CenteredLayer类通过继承Layer类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了call函数里。这个层里不含模型参数。

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential

In [0]:
tf.enable_eager_execution()

In [0]:
class CenteredLayer(layers.Layer):
  def __init__(self):
    super(CenteredLayer,self).__init__()
  
  def call(self,x):
    return x-tf.reduce_mean(x)

&emsp;&emsp;我们可以实例化这个层，然后做前向计算。

In [0]:
layer=CenteredLayer()
layer(tf.constant([1,2,3,4,5]))

<tf.Tensor: id=4, shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

&emsp;&emsp;我们也可以用它来构造更复杂的模型。

In [0]:
net=Sequential()
net.add(layers.Dense(128))
net.add(CenteredLayer())

&emsp;&emsp;下面打印自定义层各个输出的均值。因为均值是浮点数，所以它的值是一个很接近0的数。

In [0]:
y=net(tf.random.uniform(shape=(4,8)))
tf.reduce_mean(y).numpy()

-3.259629e-09

###4.3.2. 含模型参数的自定义层
&emsp;&emsp;我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。我们尝试实现一个含权重参数和偏差参数的全连接层。它使用ReLU函数作为激活函数。其中in_units和units分别代表输入个数和输出个数。

In [0]:
class MyDense(layers.Layer):
  #units为该层的输出个数
  def __init__(self,units):
    super(MyDense,self).__init__()
    self.units=units

  def build(self,input_shape):
    self.kernel=self.add_weight(name='kernel',shape=[int(input_shape[-1]),self.units])

  def call(self,input):
    return tf.matmul(input,self.kernel)

&emsp;&emsp;下面，我们实例化MyDense类并访问它的模型参数。

In [0]:
mydense=MyDense(10)

In [0]:
mydense.trainable_weights

[<tf.Variable 'my_dense/kernel:0' shape=(256, 10) dtype=float32, numpy=
 array([[-0.06160554, -0.04592701,  0.03480589, ..., -0.10063919,
         -0.08592829,  0.06521817],
        [ 0.05079053,  0.00844572,  0.00859797, ..., -0.10493287,
         -0.14006785, -0.08032777],
        [ 0.06736386,  0.1280073 , -0.01667269, ..., -0.01657282,
         -0.08774073,  0.14624679],
        ...,
        [-0.0043927 ,  0.01602679, -0.06134421, ...,  0.06288308,
          0.01501806, -0.12082192],
        [ 0.14401358, -0.13923575, -0.03092163, ..., -0.03969427,
          0.08999053, -0.14942415],
        [-0.12680429,  0.11857036, -0.04953749, ..., -0.12889509,
         -0.01627132,  0.00323282]], dtype=float32)>]

&emsp;&emsp;我们可以直接使用自定义层做前向计算

In [0]:
y=mydense(tf.random.normal(shape=(5,256)))
y.shape

TensorShape([Dimension(5), Dimension(10)])

In [0]:
net=Sequential()
net.add(MyDense(8))
net.add(MyDense(1))
for layer in net.layers:
  print(layer.name)

my_dense_1
my_dense_2


In [0]:
net(tf.random.uniform(shape=(2,64)))

<tf.Tensor: id=124, shape=(2, 1), dtype=float32, numpy=
array([[ 0.00493592],
       [-0.56078476]], dtype=float32)>

###4.3.3. 小结
* 可以通过Layer类自定义神经网络中的层，从而可以被重复调用